In [1]:
"""
        SETUP ENVIRONMENT

Choose a game:
- "Cart" # ok for 100 epoch
- "Car" # not good choice
- "Pendulum" # not good choice
- "Lake" # demand more then 1500 epoch
- "Maze" # 100 epoch is OK
- "Toy" # 100 epoch
"""
GAME_NAME = "Toy"
HORIZON = 7 #fixed horizon
"""
            INITIALIZING AGENTS and ENVIRONMENT
"""
"""-------------------------------------------------------------------"""
from enviroment import *
from utils import train_agent
from utils import Ntk_analysis
env, obs_dim, action_dim = game_setup(GAME_NAME )
env.rnd_init_state = False
env.testing = True

In [2]:
"""
                 SETUP AGENT and TRAINING
"""

"""REINFORCE"""
HIDDEN_DIM_REIN = 24
EPSILON = 0.15 # for apsilon greedy policy
"""MATRYOSHKA"""
HIDDEN_DIM_MTR = HIDDEN_DIM_REIN
"""ResNet"""
HIDDEN_DIM_RESNET = 16
#MTRNet with dynamically changing parameters number per layer
INIT_HIDDEN_LAYER = 20

"""Training"""
TAU = 0.6
LEARNING_RATE = 0.001
NUM_EPOCHES = 400
N_EPISODES = 40 # number of episodes per epoch.

"""Testing"""
NUM_TEST_EPISODES = 300


"""DYNAMICAL TAU AND LR"""
PATIENCE = 20 # for dynamical tau and lr
TAU_END = TAU # for dynamical tau
LR_END = LEARNING_RATE / 2000 # for dynamical learning rate
"""
                    INITIALIZING AGENTS
"""
"""-------------------------------------------------------------------"""
#more settings---------------------
HORIZON_REIN = HORIZON
HORIZON_MTR = HORIZON
LEARNING_RATE_REIN = LEARNING_RATE
LEARNING_RATE_MTR  = LEARNING_RATE
LEARNING_RATE_RESNET = LEARNING_RATE
LEARNING_RATE_ORIGINAL = LEARNING_RATE
#-------------------------------------
HIDDEN_DIM_ORIGINAL = HIDDEN_DIM_RESNET
from original import OriginalMtrAgent
from NeuralNet import ReinforceAgent, MTRAgent
from MtrNet import ReinforceMtrNetAgent, MtrNetAgent, ShortLongAgent
import torch

#MTR with MTRNet
agent4 = MtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU)

#Original MTR
agent5 = OriginalMtrAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_ORIGINAL,horizon= HORIZON, learning_rate= LEARNING_RATE_ORIGINAL , game_name= GAME_NAME, tau=TAU)



In [3]:
"""DataFrame to store results"""
import pandas as pd
FROM_PATH = "optimality_test.csv"
TO_PATH = "optimality_test.csv"
df = pd.read_csv(FROM_PATH)
data1 = {"init": "eoc", 'description': "Toy, hor: 7. "}

# function to push data
def push(df,data2,data3,data1=data1):
    data1.update(data2)
    data1.update(data3)
    df = df._append(data1, ignore_index=True)
    return df
#load to csv file path
def load(df,path=TO_PATH):
    df.to_csv(path, index=False)

In [7]:
"""Check the optimal solution"""
env.generate_all_q_stars(HORIZON, tau =0.3)
v_star = env.v_star[str(HORIZON)]
opt_sol = v_star[0]
print(opt_sol)

Calculated until horizon 7
0.5627487176804834


In [5]:
df

,game,agent,init,lr,tau,test,train,train_entropy,full_rank,total_params,opt_sol,description,opt_sol
0,Lol,NaN,eoc,0.0000,0.0,0.000000,0.0,0,0,0,0.0,NaN,NaN
1,Toy,OriginalMtr,eoc,0.0001,0.6,0.639667,"[-0.017999999999999967, 0.17399999999999977, 0...","[-0.5984274499858732, -0.38571739031374463, -0...",1,3038,NaN,"Toy, hor: 7.",0.371557
2,Toy,OriginalMtr,eoc,0.0001,0.6,0.641333,"[0.009999999999999998, 0.062499999999999986, 0...","[-0.5156363041698933, -0.42130479626357553, -0...",1,3038,NaN,"Toy, hor: 7.",0.371557


In [30]:
"""
    Matryoshka "ORIGINAL" Training
"""

"""-------------------------------------------------------------------"""
agent5.policy.ntk_init()
agent5.tau = TAU
agent5.lr = LEARNING_RATE
_,_,data5 = train_agent(agent5, env, num_epoches=NUM_EPOCHES*4, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=agent5.tau, lr_end=LR_END, patience=PATIENCE, clip_grad=False, testing=True)
"""Testing"""
from utils import test_agent
_, data5_ = test_agent(agent5, env, GAME_NAME, n_episodes=NUM_TEST_EPISODES, tau=agent5.tau)

 sigma_w = 1.4142135623730951
 sigma_b = 0
Epoch 1
tau 0.600
Learning rate 0.90936 * 10^-3
Reward: -0.3949999999999999
Entropy Reward: -0.8330426853196696
Calculated until horizon 7
Optimal entropy solution: 0.3715566147907546
CK full rank: True
Test reward: 0.0700000000000002

Epoch 50
tau 0.600
Learning rate 0.75466 * 10^-3
Reward: 0.5975
Entropy Reward: 0.3448601762671024
Calculated until horizon 7
Optimal entropy solution: 0.3715566147907546
CK full rank: True
Test reward: 0.5700000000000001

Epoch 99
tau 0.600
Learning rate 0.62919 * 10^-3
Reward: 0.6475000000000005
Entropy Reward: 0.3651392852328719
Calculated until horizon 7
Optimal entropy solution: 0.3715566147907546
CK full rank: True
Test reward: 0.6462499999999995

Epoch 148
tau 0.600
Learning rate 0.48304 * 10^-3
Reward: 0.6350000000000005
Entropy Reward: 0.3687478426750751
Calculated until horizon 7
Optimal entropy solution: 0.3715566147907546
CK full rank: True
Test reward: 0.6500000000000006

Epoch 197
tau 0.600
Learnin

In [62]:
load(df)

In [11]:
ck = Ntk_analysis(env)
ck.full_analysis(agent5, mode = "ck")

In [ ]:
ck.rank

In [13]:
ck.genarate_optimal_Q(agent5)
ck.policy_eval()
ck.m

Calculated until horizon 4


{'1': {'overall': 0.10441570319714011,
  'max': 0.05092001038908558,
  'mean': 0.026103925799285027},
 '2': {'overall': 0.0922388014922231,
  'max': 0.05087902263453494,
  'mean': 0.023059700373055776},
 '3': {'overall': 0.07993470315002459,
  'max': 0.05345913956517403,
  'mean': 0.019983675787506146},
 '4': {'overall': 0.039458952234961764,
  'max': 0.039458952234961764,
  'mean': 0.009864738058740441}}

In [27]:
"""
    MTRNet Training
"""
"""-------------------------------------------------------------------"""
agent4.tau = 0.4
agent4.lightMTR = True
agent4.lr = LEARNING_RATE
agent4.policy.ntk_init()
_,_,data4 = train_agent(agent4, env, num_epoches=NUM_EPOCHES*3, n_episodes=N_EPISODES,game_name=GAME_NAME, tau_end=agent4.tau, lr_end=LR_END, patience=PATIENCE, clip_grad=True, testing=True)

"""Testing"""
from utils import test_agent
_, data4_ = test_agent(agent4, env, GAME_NAME, n_episodes=NUM_TEST_EPISODES, tau=agent4.tau)

 sigma_w = 1.4142135623730951
 sigma_b = 0
Epoch 1
tau 0.400
Learning rate 0.88101 * 10^-3
Reward: 0.8275000000000003
Entropy Reward: -0.34565317362081255
Calculated until horizon 7
Optimal entropy solution: 0.47971497176851524
CK full rank: False
Test reward: 0.8637500000000016

Epoch 50
tau 0.400
Learning rate 0.68815 * 10^-3
Reward: 0.6625000000000001
Entropy Reward: -0.06710230871103698
Calculated until horizon 7
Optimal entropy solution: 0.47971497176851524
CK full rank: False
Test reward: 0.6087500000000013

Epoch 99
tau 0.400
Learning rate 0.54192 * 10^-3
Reward: 0.6900000000000005
Entropy Reward: 0.19275217301677916
Calculated until horizon 7
Optimal entropy solution: 0.47971497176851524
CK full rank: True
Test reward: 0.5900000000000003

Epoch 148
tau 0.400
Learning rate 0.38433 * 10^-3
Reward: 0.5650000000000002
Entropy Reward: 0.1699173265695572
Calculated until horizon 7
Optimal entropy solution: 0.47971497176851524
CK full rank: True
Test reward: 0.5437500000000001

Epoch 

In [28]:
df =push(df, data4, data4_)

In [31]:
ck = Ntk_analysis(env)
ck.full_analysis(agent4, mode = "ck")

In [32]:
ck.rank

{'1': 8, '2': 8, '3': 8, '4': 8}

In [32]:
load(df)